In [ ]:
!python --version

In [ ]:
%pip install -q "openvino>=2024.1.0" "nncf>=2.10.0"

In [ ]:
%pip install -q "python-ffmpeg<=1.0.16" moviepy "onnx!=1.16.2" 

In [ ]:
%pip install -q "torch>=2.1" --extra-index-url https://download.pytorch.org/whl/cpu

In [ ]:
%pip install tf-keras

In [ ]:
%pip install -q "git+https://github.com/huggingface/optimum-intel.git"

In [ ]:
import torch
print(torch.__version__)

In [ ]:
import numpy as np
print(np.version.version)

In [8]:
model_id = "openai/whisper-small"

In [9]:
from pathlib import Path

model_dir = model_id.split("/")[-1]

if not Path(model_dir).exists():
    !optimum-cli export openvino -m {model_id} {model_dir} --weight-format fp16

In [10]:
import openvino as ov
core = ov.Core()
supported_devices = core.available_devices + ["AUTO"]
print(supported_devices)

In [ ]:
from optimum.intel.openvino import OVModelForSpeechSeq2Seq
from transformers import AutoProcessor, pipeline

device = 'CPU'

ov_model = OVModelForSpeechSeq2Seq.from_pretrained(model_dir, device=device)
processor = AutoProcessor.from_pretrained(model_dir)

# GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
# > It looks like it at this point, until METAL support is added, no GPU for ARM Macintoshes.
# https://github.com/black-forest-labs/flux/issues/48#issuecomment-2273668173

pipe = pipeline(
    "automatic-speech-recognition",
    model=ov_model,
    chunk_length_s=30,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
)

In [ ]:
import os
print(os.path.exists("jp-sample.wav"))

In [ ]:
import numpy as np
from scipy.io import wavfile
# we can't use the "wave" standard library because it can't read 32f files.
# in Adobe Audition you can't export lower than 32f so we'll have to covert.
# We'll use wavefile scipy.io wavfile

audio_path = "./jp-sample.wav"

# Read the 32-bit float WAV file
sampling_rate, raw = wavfile.read('./jp-sample.wav')

In [ ]:
%%time
task = 'transcribe' # transcribe / translate
transcription = pipe(
    {"raw": raw, "sampling_rate": sampling_rate }, 
    generate_kwargs={"task": task}, 
    return_timestamps=True
)["chunks"]

In [ ]:
print(transcription)